# Clustering Crypto

In [18]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [19]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [20]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
r = requests.get(url)
data = r.json()
list(data.keys())
df = pd.DataFrame(data['Data'])
df = df.transpose()
df.head()


,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,AlgorithmType
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Alternatively, use the provided csv file:
file_path = open("Resources/crypto_data.csv")
df = pd.read_csv(file_path, index_col=0)
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [22]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
# data is already cleaned in CSV FILE
df.columns

Index(['CoinName', 'Algorithm', 'IsTrading', 'ProofType', 'TotalCoinsMined',
       'TotalCoinSupply'],
      dtype='object')

In [23]:
# Keep only cryptocurrencies that are trading
df_crypto = df[df.IsTrading.eq(True)]
df_crypto.shape


(1144, 6)

In [24]:
# Keep only cryptocurrencies with a working algorithm
pd.isna(df_crypto['Algorithm'])

42      False
365     False
404     False
611     False
808     False
        ...  
SERO    False
UOS     False
BDX     False
ZEN     False
XBC     False
Name: Algorithm, Length: 1144, dtype: bool

In [25]:
# Remove the "IsTrading" column
df_crypto = df_crypto.drop(['IsTrading'], axis = 1)
df_crypto.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [26]:
# Remove rows with at least 1 null value
df_crypto = df_crypto.dropna(how='any', axis=0)
df_crypto.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [27]:
# Remove rows with cryptocurrencies having no coins mined
df_crypto = df_crypto[df_crypto.TotalCoinsMined > 0]
df_crypto.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [28]:
# Drop rows where there are 'N/A' text values
df_crypto = df_crypto[df_crypto!='N/A']
df_crypto.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [29]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinName = df_crypto.filter(['CoinName'], axis=1)
coinName.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [30]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
df_crypto = df_crypto.drop(['CoinName'], axis=1)
df_crypto.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [31]:
# Create dummy variables for text features
crypto = pd.get_dummies(df_crypto['Algorithm'])
dummy = pd.get_dummies(df_crypto['ProofType'])
combined = pd.concat([crypto, dummy], axis=1)
df = df_crypto.merge(combined, left_index=True, right_index=True)
df = df.drop(['Algorithm', 'ProofType'], axis=1)
df.head()


,TotalCoinsMined,TotalCoinSupply,1GB AES Pattern Search,536,Argon2d,BLAKE256,Blake,Blake2S,Blake2b,C11,...,PoW/PoS,PoW/PoS,PoW/PoW,PoW/nPoS,Pos,Proof of Authority,Proof of Trust,TPoS,Zero-Knowledge Proof,dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Standardize data
df_scaled = StandardScaler().fit_transform(df)
# print(df_scaled)


### Reducing Dimensions Using PCA

In [33]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)
df_pca


array([[-0.33104148,  0.93570102, -0.58573496],
       [-0.31439083,  0.93632938, -0.58602555],
       [ 2.29447393,  1.70275584, -0.67142676],
       ...,
       [ 0.32123255, -2.28038357,  0.41920867],
       [-0.15329326, -1.8994442 ,  0.41318124],
       [-0.28561044,  0.8414177 , -0.26488698]])

In [34]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(data=df_pca, columns=['PC1', 'PC2', 'PC3'], index=df_crypto.index)
pcs_df

,PC1,PC2,PC3
42,-0.331041,0.935701,-0.585735
404,-0.314391,0.936329,-0.586026
1337,2.294474,1.702756,-0.671427
BTC,-0.152382,-1.293933,0.204551
ETH,-0.159560,-1.957341,0.403593
...,...,...,...
ZEPH,2.474360,0.491837,0.066039
GAP,-0.329088,0.935647,-0.585754
BDX,0.321233,-2.280384,0.419209
ZEN,-0.153293,-1.899444,0.413181


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [35]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k":k,"inertia":inertia}
elbow = pd.DataFrame(elbow_data)
elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [36]:
# Initialize the K-Means model
kmeans = KMeans(n_clusters=4, random_state=0)
# Fit the model
kmeans.fit(pcs_df)
# Predict clusters
pred = kmeans.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([df_crypto,pcs_df],axis =1)
clustered_df['CoinName'] = coinName['CoinName']
clustered_df["Class"] = kmeans.labels_
print(clustered_df.shape)
clustered_df.head(10)


(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.331041,0.935701,-0.585735,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.314391,0.936329,-0.586026,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.294474,1.702756,-0.671427,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.152382,-1.293933,0.204551,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.159560,-1.957341,0.403593,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.161216,-1.187782,-0.032735,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.391807,1.212097,-0.479576,Dash,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.150076,-2.180496,0.350764,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.158003,-1.957391,0.403579,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.153292,-1.899444,0.413181,ZCash,0


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [37]:
# Scale data to create the scatter plot
fig = px.scatter_3d(
    clustered_df,
    hover_name="CoinName",
    hover_data=["Algorithm"],
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [38]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'].astype(
    float) / 1000000
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(
    float) / 1000000
clustered_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply",
                            by="Class", colormap="viridis", hover_cols=["CoinName"])


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [39]:
# Table with tradable cryptos
columns = ['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply', 'TotalCoinsMined', 'Class']


In [40]:
# Print the total number of tradable cryptocurrencies
clustered_df.hvplot.table(columns,sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]